<a href="https://colab.research.google.com/github/Bae-hong-seob/Android_studio_app/blob/main/Big_data_process_and_application/4_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 51.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ffc990f5e1479e9726345db132de58233d5e17f44ee6b13de89e2cf7f6358427
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [ ]:
!pip install graphframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 6.2 MB/s 


In [ ]:
 !curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes- 0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark- packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2- spark3.2-s_2.12.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server
curl: (6) Could not resolve host: packages
curl: (6) Could not resolve host: spark3.2-s_2.12.jar


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq

In [ ]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf

In [ ]:
from pyspark.sql.types import *
from graphframes import *

In [ ]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()
spark

### 무향 그래프 -> 양방향 그래프로 변환

In [ ]:
def create_transport_graph():
  node_fields = [
      StructField("id", StringType(), True),
      StructField("latitude", StringType(), True),
      StructField("longitude", StringType(), True),
      StructField("population", StringType(), True),
  ]

  nodes = spark.read.csv("/content/drive/MyDrive/3-2 university/Big_data_process_and_application/input/transport-nodes.csv", header=True, schema=StructType(node_fields))

  rels = spark.read.csv("/content/drive/MyDrive/3-2 university/Big_data_process_and_application/input/transport-relationships.csv", header=True)
  reversed_rels = (rels.withColumn('newSrc', rels.dst)).withColumn("newDst",rels.src).drop("dst","src").withColumnRenamed("newSrc","src").withColumnRenamed("newDst","dst").select("src","dst","relationship","cost")

  relationship = rels.union(reversed_rels)

  return GraphFrame(nodes, relationship)

In [ ]:
g = create_transport_graph()

# BFS 진행

필요한건 무조건 start!

In [ ]:
(g.vertices
 .filter("population > 100000 and population < 300000")
 .sort("population")
 .show())

+----------+--------+---------+----------+
|        id|latitude|longitude|population|
+----------+--------+---------+----------+
|Colchester|51.88921|  0.90421|    104390|
|   Ipswich|52.05917|  1.15545|    133384|
+----------+--------+---------+----------+



In [ ]:
from_expr = "id = 'Den Haag'"
to_expr = "population > 100000 and population < 300000 and id <> 'Den Haag'"
result = g.bfs(from_expr,to_expr)
print(result.columns)

['from', 'e0', 'v1', 'e1', 'v2', 'e2', 'to']


/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
columns = [columns for columns in result.columns if not columns.startswith("e")]
result.select(columns).show()

+--------------------+--------------------+--------------------+--------------------+
|                from|                  v1|                  v2|                  to|
+--------------------+--------------------+--------------------+--------------------+
|{Den Haag, 52.078...|{Hoek van Holland...|{Felixstowe, 51.9...|{Ipswich, 52.0591...|
+--------------------+--------------------+--------------------+--------------------+

